In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [17]:
class GNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)
        
    def forward(self,data):
        x, edge_index = data.x, data.edge_index
        
        x=self.conv1(x,edge_index)
        x=F.relu(x)
        x=F.dropout(x, training=self.training)
        x=self.conv2(x,edge_index)
        
        return F.log_softmax(x, dim=1)

In [5]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='F://dataset', name='Cora')

Processing...
Done!


In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cup')
model = GNN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [20]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [26]:
model.train()
for epoch in range(1000):
    optimizer.zero_grad()
    out=model(data)
    loss=F.nll_loss(out[data.train_mask],data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [27]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct)/int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.8110
